In [1]:
import pandas as pd
import numpy as np
import itertools

# 2. 시각화
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# 3. 유틸
# tqdm 패키지는 반복문에 대해 얼마나 진척되었는지를 가시적으로 확인할 수 있도록 도와줍니다.
# https://github.com/tqdm/tqdm 사용법은 정말 간단합니다.
from tqdm.auto import tqdm


# 4. 설정
# 경고가 나와서, 출력이 많아지지 않기 위해 ignore를 설정해주었습니다.
import warnings
warnings.filterwarnings('ignore')


# 5. stats models
# 시계열 모델을 위한 ARIMA를 임포트 해주었습니다.
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

In [21]:
test_x = pd.read_csv("test_x_df.csv")


In [2]:
train_x = pd.read_csv("train_x_df.csv")
train_y = pd.read_csv("train_y_df.csv")

In [3]:
def make_mfi(my_df):
    period = 14
    result = []

    for sample_id in my_df['sample_id'].unique().tolist():
      df = my_df[my_df['sample_id'] == sample_id]

      df['typical_price'] = (df['close'] + df['high'] + df['low']) / 3
      df['money_flow'] = df['typical_price'] * df['volume']
      df['price_diff'] = df.groupby(['sample_id'])['typical_price'].diff(1).shift(-1)
      df['pf'] = np.where(df['price_diff'] > 0, df['money_flow'], 0)
      df['nf'] = np.where(df['price_diff'] < 0, df['money_flow'], 0)

      # 상승분의 14일 평균을 구해줍니다.
      # 하락분의 14일 평균을 구해줍니다.
      df["pmf"] = df["pf"].rolling(window=period, min_periods=period).sum()
      df["nmf"] = df["nf"].rolling(window=period, min_periods=period).sum()

      MFI = df['pmf'] / (df['pmf'] + df['nmf'])
      df['mfi'] = MFI
      df['mfi'] = df['mfi'].shift(1)

      result.append(df)
      
    output = pd.concat(result, axis=0)

    return output

In [22]:
test_x = make_mfi(test_x)

In [ ]:
test_x

In [4]:
train_x = make_mfi(train_x[train_x['sample_id'] < 1000])
train_y = make_mfi(train_y[train_y['sample_id'] < 1000])

In [43]:
train_x

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,typical_price,money_flow,price_diff,pf,nf,pmf,nmf,mfi
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614,0.993123,1369.991487,-0.000133,0.000000,1369.991487,NaN,NaN,NaN
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695,0.992990,3414.701802,0.001039,3414.701802,0.000000,NaN,NaN,NaN
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721,0.994030,3692.769439,0.000761,3692.769439,0.000000,NaN,NaN,NaN
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776,0.994791,2417.605282,-0.000363,0.000000,2417.605282,NaN,NaN,NaN
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785,0.994428,3045.078194,0.000520,3045.078194,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551995,399,1375,6,1.006870,1.008103,1.006341,1.006518,21.015982,1201.546631,2.060948,11.966638,684.212280,1.006987,21.162826,-0.002936,0.000000,21.162826,114.031268,149.229067,0.482678
551996,399,1376,6,1.006518,1.006518,1.002642,1.002995,29.148287,1661.261963,2.871235,9.344916,532.389954,1.004051,29.266378,-0.003347,0.000000,29.266378,114.031268,176.229006,0.433150
551997,399,1377,6,1.002995,1.002995,0.996829,1.002290,88.265602,5008.012695,10.551348,34.899464,1979.980103,1.000705,88.327796,0.000528,88.327796,0.000000,202.359065,173.961499,0.392859
551998,399,1378,6,1.002114,1.003347,1.000176,1.000176,28.551916,1624.406128,3.434913,13.287203,755.970886,1.001233,28.587120,-0.004932,0.000000,28.587120,202.359065,182.177143,0.537731


In [24]:
TEST_SAMPLE_ID_LIST = test_x["sample_id"].unique().tolist()

In [5]:
TRAIN_SAMPLE_ID_LIST = train_x["sample_id"].unique().tolist()

In [6]:
def get_typical_price(df,sample_id):    
    return df[df["sample_id"] == sample_id]['typical_price'].values

In [7]:
def get_mfi(df,sample_id):
  return df[df["sample_id"] == sample_id]['mfi'].values

In [8]:
# AIC 값이 최소인 p,d,q 값 직접 구하기
def model_fit(df, sample_id_list):
    result = []

    for sample_id in tqdm(sample_id_list):

        price_x = get_typical_price(df, sample_id)
        mfi_series = get_mfi(df, sample_id)

        # 2. ARIMA
        # 1) 모델 정의
        ARIMA_MODEL = {}
        ARIMA_MODEL_FIT = {}

        # 2) AR 모델 적용
        try:
            ARIMA_MODEL = ARIMA(price_x, order = (5,1,1))
            ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

        # 3) 수렴하지 않을 경우 p d q 를 1, 1, 0으로 사용
        except:
            ARIMA_MODEL = ARIMA(price_x, order = (1,1,0))
            ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

        # 4) ARIMA 예측
        ARIMA_FORECAST  = ARIMA_MODEL_FIT.predict(1,120, typ='levels')

        # 3. 데이처 처리
        # 1) 최대 부분인 인덱스를 찾는데 해당 시점에 매도를 진행합니다.
        sell_time = np.argmax(ARIMA_FORECAST)

        # 2) 최대값을 찾습니다.
        max_val = np.max(ARIMA_FORECAST)

        mfi_last_val = mfi_series[1379]

        # 4. 투자 전략
        buy_quantity = 0

        # 1) typical_price가 1.1 이상이면 투자합니다.
        if  max_val > 1.15:
            buy_quantity = 1

        if mfi_last_val > 0.7:
            buy_quantity = 0
            
        # if mfi_last_val < 0.2:
        #     buy_quantity = 1

        # 5. 결과
        result_list = [
                        sample_id,
                        buy_quantity,
                        sell_time,
                        max_val,
                    ]

        result.append(result_list)

    return result

In [9]:
# result = model_fit(test_x, TEST_SAMPLE_ID_LIST)
result = model_fit(train_x, TRAIN_SAMPLE_ID_LIST)

  3%|▎         | 30/1000 [00:12<06:15,  2.58it/s]C:\Users\user1\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\user1\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
100%|██████████| 1000/1000 [07:53<00:00,  2.11it/s]


In [10]:
submit_columns = [
                  "sample_id", 
                  "buy_quantity", 
                  "sell_time",
                  "max_val"
                  ]

submit = pd.DataFrame(data=result, columns=submit_columns)

In [11]:
#투자할 sample_id 갯수 확인
submit[submit["buy_quantity"] == 1].shape[0]  

25

In [28]:
submit.buy_quantity.value_counts()

0    731
1     29
Name: buy_quantity, dtype: int64

In [12]:
sample_id = submit[submit["buy_quantity"] == 1]['sample_id']
sample_id

4        4
30      30
33      33
141    141
156    156
209    209
307    307
314    314
431    431
497    497
548    548
558    558
596    596
639    639
676    676
746    746
800    800
823    823
831    831
843    843
880    880
899    899
909    909
982    982
987    987
Name: sample_id, dtype: int64

In [15]:
for i in sample_id:
    dfy = train_y[train_y['sample_id'] == i]
    sell_time = submit[submit['sample_id'] == i]['sell_time'].values[0]
    print(dfy[dfy['time'] == sell_time]['close'].values[0], end=' ')
    val = submit[submit['sample_id'] == i]['max_val'].values[0]
    print(val)

0.9852941036224364 1.2198848844762473
1.0009417533874512 1.1659874825535426
0.9818181991577148 1.181508283997616
0.9882226586341858 1.1500890994380981
0.9857827425003052 1.194095507675599
0.97020024061203 1.2293137430288938
1.0001453161239624 1.193583226590979
1.001083493232727 1.2179777224858601
1.014967441558838 1.161400494127384
1.0131964683532717 1.249160041214169
0.997200846672058 1.1717625602320618
0.9918144345283508 1.217254151519984
0.9885765314102172 1.1659891974480512
1.0081809759140017 1.1727431025951687
1.0100773572921753 1.1752262979835224
0.9926474690437316 1.16794633755896
0.9260563254356384 2.067502770510632
0.9921023845672609 1.24185250191781
1.0098665952682495 1.1520188794281685
0.9964907765388488 1.162158907338475
0.9622569680213928 1.1519652323716778
0.9955139756202698 1.1854666073703954
0.9374434351921082 1.239370409961721
0.9987952709197998 1.2618549664815266
1.0012764930725098 1.3789609965276524


In [29]:
def df2d_to_answer(df_2d):
    # valid_y_df로부터
    # open 가격 정보가 포함된
    # [샘플 수, 120분] 크기의 
    # 2차원 array를 반환하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_2d = df_2d.open.values.reshape([sample_size, time_size])
    sample_index = list(sample_index)
    return array_2d, sample_index


def COIN(y_df, submission, df2d_to_answer = df2d_to_answer):
    # 2차원 데이터프레임에서 open 시점 데이터만 추출하여 array로 복원
    # sample_id정보를 index에 저장
    y_array, index = df2d_to_answer(y_df)

    
    # index 기준으로 submission을 다시 선택
    submission = submission.set_index(submission.columns[0])
    submission = submission.iloc[index, :]    
    
    # 초기 투자 비용은 10000 달러
    total_momey      = 10000 # dolors
    total_momey_list = []
    
    # 가장 처음 sample_id값
    start_index = submission.index[0]
    for row_idx in submission.index:
        sell_time  = submission.loc[row_idx, 'sell_time']
        buy_price  = y_array[row_idx - start_index, 0]
        sell_price = y_array[row_idx - start_index, sell_time]
        buy_quantity = submission.loc[row_idx, 'buy_quantity'] * total_momey
        residual = total_momey - buy_quantity
        ratio = sell_price / buy_price
        total_momey = buy_quantity * ratio * 0.9995 * 0.9995 + residual        
        total_momey_list.append(total_momey)
        
    return total_momey, total_momey_list

In [38]:
FILE_NAME = "/0617_AUTO_ARIMA_MFI_20UNDER_MAXVAL_110_SUBMIT.csv"

In [39]:
SUBMIT_PATH = "./data"
RESULT_PATH = SUBMIT_PATH + FILE_NAME

submit.to_csv(RESULT_PATH, index=False)